In [ ]:
# Needs in terminal:
# $ quilc -S
# $ qvm -S

import numpy as np

from pyquil.api import get_benchmarker, get_qc
from forest_benchmarking.rb import rb_dataframe, \
    add_sequences_to_dataframe, \
    run_rb_measurement, \
    add_survivals, \
    survivals_by_qubits, \
    fit_standard_rb
from forest_benchmarking.analysis.fitting import make_figure

from pyquil.gates import *
from pyquil import Program


%matplotlib inline

# You need a Quantum Computer and a Benchmarker

In [ ]:
bm = get_benchmarker()
qc = get_qc("9q-square-qvm")

# Setup Both RB experiments

In [ ]:
# Choose a subgraph
subgraph = [(0, 1)]
# Intialize and partially populate a DataFrame to summarize the experiment.
df = rb_dataframe(rb_type="sim-2q",
                  subgraph=subgraph,
                  depths=3 * 2 ** np.arange(4, dtype=np.uint8),
                  num_sequences=25)
# Generate standard RB sequences in new dataframe. This is slow!
standard_rb_df = add_sequences_to_dataframe(df, bm)

# Run Standard RB experiment

In [ ]:
# Generate standard RB sequences in new dataframe. This is slow!
standard_rb_df = add_sequences_to_dataframe(df, bm)

# Run the RB Sequences on a QuantumComputer
standard_rb_df = run_rb_measurement(standard_rb_df, qc, num_trials=100)

# Calculate survival statistics and add these to the DataFrame
standard_rb_df = add_survivals(standard_rb_df)

# Extract the survival data into arrays keyed to each measured qubit or qubit-pair
depths, survivals, survival_errs = {}, {}, {}
for qubits in subgraph:
    depths[qubits], survivals[qubits], survival_errs[qubits] = survivals_by_qubits(standard_rb_df, qubits)
    
# Fit the data
fit = fit_standard_rb(depths[subgraph[0]], survivals[subgraph[0]], weights=1/survival_errs[subgraph[0]])

# Plot
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")

# Extract rb decay parameter
rb_decay = fit.params['decay'].value

# Run Interleaved RB experiment

In [ ]:
# Generate interleaved RB sequences in new dataframe. This is slow!
irb_df = add_sequences_to_dataframe(df, bm, interleaved_gate=Program(CNOT(0,1)))

# Run the RB Sequences on a QuantumComputer
irb_df = run_rb_measurement(irb_df, qc, num_trials=100)

# Calculate survival statistics and add these to the DataFrame
irb_df = add_survivals(irb_df)

# Extract the survival data into arrays keyed to each measured qubit or qubit-pair
depths, survivals, survival_errs = {}, {}, {}
for qubits in subgraph:
    depths[qubits], survivals[qubits], survival_errs[qubits] = survivals_by_qubits(irb_df, qubits)
    
# Fit the data
fit = fit_standard_rb(depths[subgraph[0]], survivals[subgraph[0]], weights=1/survival_errs[subgraph[0]])

# Plot
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")

# Extract rb decay parameter
irb_decay = fit.params['decay'].value


In [ ]:
print(rb_decay)
print(irb_decay)

# Average Clifford gate fidelity, and interleaved gate fidelity

In [ ]:
from forest_benchmarking.rb import RB_decay_to_gate_fidelity, \
    irb_decay_to_gate_infidelity, \
    interleaved_gate_fidelity_bounds, \
    strip_inverse_from_sequences

In [ ]:
print(RB_decay_to_gate_fidelity(rb_decay, 4))
gate_fidelity = 1 - irb_decay_to_gate_infidelity(irb_decay, rb_decay, 4)
print(gate_fidelity)

# Interleaved gate fidelity bounds

In [ ]:
bounds = interleaved_gate_fidelity_bounds(irb_decay, rb_decay, 4)
print(bounds)

In [ ]:
assert(bounds[0] < gate_fidelity and gate_fidelity < bounds[1])

# Improve gate fidelity bounds with unitarity experiment. EXTREMELY SLOW

In [ ]:
from forest_benchmarking.rb import run_unitarity_measurement, \
    add_shifted_purities, \
    shifted_purities_by_qubits, \
    fit_unitarity

In [ ]:
num_trials_per_seq = 25

# populate dataframe with each sequence 
unitarity_df = strip_inverse_from_sequences(standard_rb_df)

# run num_trials_per_sequence indepedent measurements on the qc 
# for each sequence in the dataframe
unitarity_df = run_unitarity_measurement(unitarity_df, qc, num_trials = num_trials_per_seq)

# calculate and store purity statistics from the measurement results
unitarity_df = add_shifted_purities(unitarity_df) 

# organize the statistics by the qubit(s) components in the subgraph (here only one)
depths, purities, purity_errs = {}, {}, {} 
for qubits in subgraph:
    depths[qubits], purities[qubits], purity_errs[qubits] = shifted_purities_by_qubits(unitarity_df, qubits)

# fit a model for the first (and only) component in the subgraph
fit = fit_unitarity(depths[subgraph[0]], purities[subgraph[0]], weights= 1/purity_errs[subgraph[0]])

# plot the raw data, point estimate error bars, and fit
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Shifted Purity")
unitarity = fit.params['unitarity'].value

## This may result in NaN depending on the outcome of the unitarity and difference between rb and irb decays. Getting better estimates of each helps prevent this

In [ ]:
better_bounds = interleaved_gate_fidelity_bounds(irb_decay, rb_decay, 4, unitarity)
print(better_bounds)